In [94]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import requests

In [95]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?fish ?fishLabel ?wormsLabel ?pic
WHERE {
  ?fish wdt:P171* wd:Q27207.
  ?fish wdt:P105 wd:Q7432.
  ?fish wdt:P850 ?worms.
  ?fish wdt:P18 ?pic.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 100
"""

In [96]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [97]:
results = get_results(endpoint_url, query)

In [98]:
for row in results["results"]["bindings"]:
    for key in row:
        row[key] = row[key]["value"]

In [99]:
df = pd.DataFrame.from_dict(results["results"]["bindings"])

In [100]:
df

,fish,pic,fishLabel,wormsLabel
0,http://www.wikidata.org/entity/Q1887880,http://commons.wikimedia.org/wiki/Special:File...,Sixspine leatherjacket,220060
1,http://www.wikidata.org/entity/Q1901712,http://commons.wikimedia.org/wiki/Special:File...,Smooth leatherjacket,277614
2,http://www.wikidata.org/entity/Q1954812,http://commons.wikimedia.org/wiki/Special:File...,Monacanthus ciliatus,159497
3,http://www.wikidata.org/entity/Q1954812,http://commons.wikimedia.org/wiki/Special:File...,Monacanthus ciliatus,159497
4,http://www.wikidata.org/entity/Q2078547,http://commons.wikimedia.org/wiki/Special:File...,Slender Filefish,159499
...,...,...,...,...
95,http://www.wikidata.org/entity/Q1687707,http://commons.wikimedia.org/wiki/Special:File...,Buenos Aires tetra,1016061
96,http://www.wikidata.org/entity/Q1812367,http://commons.wikimedia.org/wiki/Special:File...,Hyphessobrycon peruvianus,1014975
97,http://www.wikidata.org/entity/Q2002023,http://commons.wikimedia.org/wiki/Special:File...,Hyphessobrycon columbianus,1384285
98,http://www.wikidata.org/entity/Q2606177,http://commons.wikimedia.org/wiki/Special:File...,Clown barb,1461347


In [101]:
WORMS_BASE_URL = "https://www.marinespecies.org/rest/"
ATTRIBUTE_BY_ID = "AphiaAttributesByAphiaID/{id}"

In [114]:
enrichment = []

for i in range(len(df)):
    aphiaId = df.iloc[i]["wormsLabel"]
    res = requests.get(url=WORMS_BASE_URL + ATTRIBUTE_BY_ID.format(id=aphiaId))
    try:
        res.raise_for_status()
        for data in res.json():

            enrichment.append(data)
    except:
        print(f"AphiaId {df.iloc[i]["wormsLabel"]} failed to fetch")

AphiaId 1476251 failed to fetch
AphiaId 1476251 failed to fetch
AphiaId 1016061 failed to fetch
AphiaId 1461347 failed to fetch


In [115]:
worms_df = pd.DataFrame.from_dict(enrichment).set_index(
    ["AphiaID", "measurementTypeID"]
)

In [116]:
worms_df

measurementType  \
AphiaID measurementTypeID                                                     
220060  15                                                        Body size   
        144                Species exhibits underwater soniferous behaviour   
        23                                    Species importance to society   
277614  23                                    Species importance to society   
        15                                                        Body size   
...                                                                     ...   
1014975 23                                    Species importance to society   
1384285 144                Species exhibits underwater soniferous behaviour   
        23                                    Species importance to society   
1013917 144                Species exhibits underwater soniferous behaviour   
        23                                    Species importance to society   

                                                            measurementValue  \
AphiaID measurementTypeID                                                      
220060  15                                                                60   
        144                Likely to produce sound under natural conditio...   
        23                                                     IUCN Red List   
277614  23                 FAO-ASFIS: Species for Fishery Statistics Purp...   
        15                                                                31   
...                                                                      ...   
1014975 23                                                     IUCN Red List   
1384285 144                Likely to produce sound under natural conditio...   
        23                                                     IUCN Red List   
1013917 144                Likely to produce sound under natural conditio...   
        23                                                     IUCN Red List   

                           source_id  \
AphiaID measurementTypeID              
220060  15                    232813   
        144                   452075   
        23                    127093   
277614  23                    197354   
        15                    232813   
...                              ...   
1014975 23                    127093   
1384285 144                   452075   
        23                    127093   
1013917 144                   452075   
        23                    127093   

                                                                   reference  \
AphiaID measurementTypeID                                                      
220060  15                 May, J.L.; Maxwell, J.G.H. (1986). Trawl fish ...   
        144                Rice, A. N.; Farina, S. C.; Makowski, A. J.; K...   
        23                               IUCN Red List of Threatened Species   
277614  23                 FAO Fishery Fact Sheets Collections: Aquatic S...   
        15                 May, J.L.; Maxwell, J.G.H. (1986). Trawl fish ...   
...                                                                      ...   
1014975 23                               IUCN Red List of Threatened Species   
1384285 144                Rice, A. N.; Farina, S. C.; Makowski, A. J.; K...   
        23                               IUCN Red List of Threatened Species   
1013917 144                Rice, A. N.; Farina, S. C.; Makowski, A. J.; K...   
        23                               IUCN Red List of Threatened Species   

                          qualitystatus  AphiaID_Inherited  CategoryID  \
AphiaID measurementTypeID                                                
220060  15                      checked             220060         NaN   
        144                  unreviewed             220060        55.0   
        23                   unreviewed             220060        13.0   
277614  23                   unreviewed             320098 

In [117]:
worms_df["measurementType"].unique()

array(['Body size', 'Species exhibits underwater soniferous behaviour',
       'Species importance to society'], dtype=object)

In [118]:
df.merge(worms_df, left_on="wormsLabel", right_on="AphiaID").drop(columns=["wormsLabel"])

,fish,pic,fishLabel,measurementType,measurementValue,source_id,reference,qualitystatus,AphiaID_Inherited,CategoryID,children
0,http://www.wikidata.org/entity/Q1887880,http://commons.wikimedia.org/wiki/Special:File...,Sixspine leatherjacket,Body size,60,232813,"May, J.L.; Maxwell, J.G.H. (1986). Trawl fish ...",checked,220060,NaN,"[{'AphiaID': '220060', 'measurementTypeID': 17..."
1,http://www.wikidata.org/entity/Q1887880,http://commons.wikimedia.org/wiki/Special:File...,Sixspine leatherjacket,Species exhibits underwater soniferous behaviour,Likely to produce sound under natural conditio...,452075,"Rice, A. N.; Farina, S. C.; Makowski, A. J.; K...",unreviewed,220060,55.0,[]
2,http://www.wikidata.org/entity/Q1887880,http://commons.wikimedia.org/wiki/Special:File...,Sixspine leatherjacket,Species importance to society,IUCN Red List,127093,IUCN Red List of Threatened Species,unreviewed,220060,13.0,"[{'AphiaID': '220060', 'measurementTypeID': 1,..."
3,http://www.wikidata.org/entity/Q1901712,http://commons.wikimedia.org/wiki/Special:File...,Smooth leatherjacket,Species importance to society,FAO-ASFIS: Species for Fishery Statistics Purp...,197354,FAO Fishery Fact Sheets Collections: Aquatic S...,unreviewed,320098,13.0,"[{'AphiaID': '277614', 'measurementTypeID': 24..."
4,http://www.wikidata.org/entity/Q1901712,http://commons.wikimedia.org/wiki/Special:File...,Smooth leatherjacket,Body size,31,232813,"May, J.L.; Maxwell, J.G.H. (1986). Trawl fish ...",checked,277614,NaN,"[{'AphiaID': '277614', 'measurementTypeID': 17..."
...,...,...,...,...,...,...,...,...,...,...,...
292,http://www.wikidata.org/entity/Q1812367,http://commons.wikimedia.org/wiki/Special:File...,Hyphessobrycon peruvianus,Species importance to society,IUCN Red List,127093,IUCN Red List of Threatened Species,unreviewed,1014975,13.0,"[{'AphiaID': '1014975', 'measurementTypeID': 1..."
293,http://www.wikidata.org/entity/Q2002023,http://commons.wikimedia.org/wiki/Special:File...,Hyphessobrycon columbianus,Species exhibits underwater soniferous behaviour,Likely to produce sound under natural conditio...,452075,"Rice, A. N.; Farina, S. C.; Makowski, A. J.; K...",unreviewed,1384285,55.0,[]
294,http://www.wikidata.org/entity/Q2002023,http://commons.wikimedia.org/wiki/Special:File...,Hyphessobrycon columbianus,Species importance to society,IUCN Red List,127093,IUCN Red List of Threatened Species,unreviewed,1384285,13.0,"[{'AphiaID': '1384285', 'measurementTypeID': 1..."
295,http://www.wikidata.org/entity/Q3546845,http://commons.wikimedia.org/wiki/Special:File...,Hyphessobrycon bentosi,Species exhibits underwater soniferous behaviour,Likely to produce sound under natural conditio...,452075,"Rice, A. N.; Farina, S. C.; Makowski, A. J.; K...",unreviewed,1013917,55.0,[]
